# OPSI Ontology Matching Pipeline

## Setup & Config

### Import required packages

In [155]:
from rdflib import Graph, URIRef, RDFS, RDF, OWL, Literal, Namespace
import os
import json
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import yaml
from typing import List, Dict, Optional
from sentence_transformers import SentenceTransformer
from rapidfuzz import fuzz


#### Load paths and constans from configuration

In [ ]:
SOURCE_PATH = "notebook_defibrilatorji/source/trbovlje_target.json"
SOURCE_INDEX_PATH = "notebook_defibrilatorji/source/source_index.faiss"
SOURCE_ID_TRACKER_PATH = "notebook_defibrilatorji/source/source_tracker.faiss"
TARGET_PATH = "notebook_defibrilatorji/target/defibrilatorji_target.json"
TARGET_INDEX_PATH = "notebook_defibrilatorji/target/target_index.faiss"
TARGET_ID_TRACKER_PATH = "notebook_defibrilatorji/target/target_tracker.faiss"
OUTPUT_PATH = "notebook_defibrilatorji/result/result_defibrilatorji.json"
EMBEDDING_MODEL_NAME = "intfloat/multilingual-e5-large"
TOP_K = 3
HCB_ENABLED = True

#### Utils functions for ontologies

In [157]:
def load_json(path):
    with open(path, encoding="utf-8") as f:
        return json.load(f)

In [158]:
def load_graph(path):
    graph = Graph()
    graph.parse(path)
    return graph

In [159]:
def get_label(graph, uri):
    label = graph.value(uri, RDFS.label)
    return str(label) if isinstance(label, Literal) else None

In [160]:
def extract_related_uris(graph, subject, predicate):
    """Dereferences URIs linked by the predicate and returns their rdfs:label."""
    values = []
    for obj in graph.objects(subject, predicate):
        label = get_label(graph, obj)
        if label:
            values.append(label)
    return values

In [161]:
def extract_superclass_labels(graph, subject):
    """Get human-readable labels of direct superclasses."""
    super_labels = []
    for superclass in graph.objects(subject, RDFS.subClassOf):
        if isinstance(superclass, URIRef):
            label = get_label(graph, superclass)
            if label:
                super_labels.append(label)
        elif (superclass, RDF.type, OWL.Restriction) in graph:
            filler = graph.value(superclass, OWL.someValuesFrom)
            if isinstance(filler, URIRef):
                super_labels.append(str(filler).split("#")[-1])
    return super_labels

#### Utils functions for generating terms JSON

In [162]:
def build_text_for_embedding(label, definition=None, synonyms=None, superclasses=None):
    parts = [f"Concept: {label}"]

    if synonyms:
        parts.append(f"Also known as: {', '.join(synonyms)}")

    if superclasses:
        parts.append(f"Part of: {', '.join(superclasses)}")

    if definition:
        parts.append(f"Defined as: {definition}")

    return ". ".join(parts)

In [163]:
def extract_enriched_terms(graph):
    terms = []
    for s in graph.subjects(RDF.type, OWL.Class):
        label = get_label(graph, s)
        if not label:
            continue

        definition = extract_related_uris(graph, s, OBO.hasDefinition)
        synonyms = extract_related_uris(graph, s, OBO.hasRelatedSynonym)
        superclasses = extract_superclass_labels(graph, s)

        enriched_text = build_text_for_embedding(
            label=label,
            definition=definition[0] if definition else None,
            synonyms=synonyms,
            superclasses=superclasses
        )

        terms.append({
            "uri": str(s),
            "label": label,
            "definition": definition[0] if definition else "",
            "synonyms": synonyms,
            "superclasses": superclasses,
            "text_for_embedding": enriched_text
        })

    return terms

In [164]:
def normalize_embeddings(embeddings: np.ndarray) -> np.ndarray:
    return embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

In [165]:
def ontology_indexing(ontology_json, faiss_index_path, id_tracker_json):

    with open(ontology_json, "r", encoding="utf-8") as f:
            ontology_terms = json.load(f)

    texts = []
    ids = []
    valid_terms = []

    for i, term in enumerate(ontology_terms):
        text = term.get("text_for_embedding")
        if not text:
            raise ValueError("Missing 'text_for_embedding'")
        texts.append(text)
        ids.append(abs(hash(term["uri"])) % (10**12))
        valid_terms.append(term)

    # Embedding
    model = SentenceTransformer(EMBEDDING_MODEL_NAME)
    embeddings = model.encode(texts, batch_size=16, show_progress_bar=True)
    embeddings = normalize_embeddings(np.array(embeddings))

    # FAISS indexing
    dimension = embeddings.shape[1]
    base_index = faiss.IndexFlatIP(dimension)
    index = faiss.IndexIDMap(base_index)
    index.add_with_ids(embeddings, np.array(ids))
    os.makedirs(os.path.dirname(faiss_index_path), exist_ok=True)
    faiss.write_index(index, faiss_index_path)

    # Save ID tracker
    id_map = {str(id_): term for id_, term in zip(ids, valid_terms)}
    with open(id_tracker_json, "w", encoding="utf-8") as f:
        json.dump(id_map, f, indent=4, ensure_ascii=False)

#### Utils for matching

In [166]:
def rerank_by_label_similarity(source_label: str, candidates: List[Dict], weight_faiss: float = 0.7, weight_label: float = 0.3) -> List[Dict]:
    """Combines semantic score and lexical similarity to rerank matches."""
    reranked = []
    for match in candidates:
        label_sim = fuzz.ratio(source_label, match["label"]) / 100
        combined_score = weight_faiss * match["score"] + weight_label * label_sim
        reranked.append({**match, "combined_score": combined_score})
    return sorted(reranked, key=lambda x: x["combined_score"], reverse=True)

In [167]:
def build_index_lookup(index_path: str, id_tracker_path: str):
    index = faiss.read_index(index_path)
    with open(id_tracker_path, "r", encoding="utf-8") as f:
        id_map = {int(k): v for k, v in json.load(f).items()}
    return index, id_map

In [168]:
def faiss_batch_search(embeddings: np.ndarray, index, top_k: int):
    return index.search(embeddings, top_k)

In [169]:
def precompute_reverse_matches(
    target_terms: List[Dict],
    reverse_index,
    reverse_id_map: Dict[int, Dict],
    model,
    top_k: int = 1
) -> Dict[str, str]:
    """
    Computes best reverse matches (target → source).
    Returns dict: target_uri → best source_uri.
    """
    reverse_lookup = {}

    valid_terms = [t for t in target_terms if t.get("text_for_embedding")]
    texts = [t["text_for_embedding"] for t in valid_terms]
    uris = [t["uri"] for t in valid_terms]

    # Batch encoding
    embeddings = model.encode(texts, batch_size=32, show_progress_bar=True)
    embeddings = normalize_embeddings(np.array(embeddings).astype(np.float32))

    # Batch FAISS search
    D, I = reverse_index.search(embeddings, top_k)

    for i, (indices, scores) in enumerate(zip(I, D)):
        best_idx = indices[0]
        if best_idx == -1:
            continue
        match = reverse_id_map.get(best_idx)
        if match:
            reverse_lookup[uris[i]] = match["uri"]

    return reverse_lookup


In [170]:
def precompute_reverse_matches_topk(
    target_terms: list,
    reverse_index,
    reverse_id_map: dict,
    model,
    top_k: int = 5
) -> dict:
    """
    Computes top-k reverse matches (target → source).
    Returns dict: target_uri → list of source_uris.
    """
    reverse_lookup_k = {}

    valid_terms = [t for t in target_terms if t.get("text_for_embedding")]
    texts = [t["text_for_embedding"] for t in valid_terms]
    uris = [t["uri"] for t in valid_terms]

    # Batch encoding
    embeddings = model.encode(texts, batch_size=32, show_progress_bar=True)
    embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)

    # Batch FAISS search
    D, I = reverse_index.search(np.array(embeddings).astype(np.float32), top_k)

    for i, indices in enumerate(I):
        matches = [reverse_id_map[idx]["uri"] for idx in indices if idx != -1 and reverse_id_map.get(idx)]
        reverse_lookup_k[uris[i]] = matches

    return reverse_lookup_k

In [171]:
def embed_terms(terms: List[Dict], model) -> np.ndarray:
    texts = [t["text_for_embedding"] for t in terms if t.get("text_for_embedding")]
    return normalize_embeddings(model.encode(texts, batch_size=32, show_progress_bar=True))


In [172]:
def map_faiss_results(source_terms, D, I, target_id_map):
    matches = []
    for i, (distances, indices) in enumerate(zip(D, I)):
        src = source_terms[i]
        results = []
        for idx, score in zip(indices, distances):
            if idx == -1:
                continue
            t = target_id_map.get(idx)
            if not t:
                continue
            results.append({
                "uri": t["uri"],
                "label": t["label"],
                "score": float(score)
            })
        matches.append({
            "source_uri": src["uri"],
            "source_label": src["label"],
            "top_k_matches": results
        })
    return matches


In [173]:
def apply_label_reranking(matches: List[Dict]):
    reranked = []
    for match in matches:
        ranked = rerank_by_label_similarity(match["source_label"], match["top_k_matches"])
        match["top_k_matches"] = ranked
        match["top_match"] = ranked[0] if ranked else None
        reranked.append(match)
    return reranked


#### Loading RDF into graph

In [174]:
source_graph = load_json("notebook_defibrilatorji/source/trbovlje_target.json")
target_graph = load_json(TARGET_PATH)

#### Loading embedding model

In [175]:
embedding_model = SentenceTransformer(EMBEDDING_MODEL_NAME)

In [176]:
target_graph

[{'uri': 'defib:zap_št',
  'label': 'ZAP ŠT.',
  'definition': 'Stolpec »ZAP ŠT.« predstavlja zaporedno številko defibrilatorja na Krškem.',
  'synonyms': [],
  'superclasses': [],
  'text_for_embedding': 'Concept: ZAP ŠT.. Type: xsd:decimal. Examples: 1.0, 2.0, 3.0. Defined as: Stolpec »ZAP ŠT.« predstavlja zaporedno številko defibrilatorja na Krškem.'},
 {'uri': 'defib:lokacija',
  'label': 'LOKACIJA',
  'definition': 'Atribut »LOKACIJA« (opisna lastnost defibrilatorja).',
  'synonyms': [],
  'superclasses': [],
  'text_for_embedding': 'Concept: LOKACIJA. Type: xsd:string. Examples: Brezje v Podboèju, Dom starejših obèanov Krško, Gasilski dom Gora. Defined as: Stolpec »LOKACIJA« predstavlja lokacijo defibrilatorja na Krškem.'},
 {'uri': 'defib:naslov',
  'label': 'NASLOV',
  'definition': 'Atribut »NASLOV« (opisna lastnost defibrilatorja).',
  'synonyms': [],
  'superclasses': [],
  'text_for_embedding': 'Concept: NASLOV. Type: xsd:string. Examples: Kje: Brezje v Podboèju 6, 8312 Pod

#### Generate enriched JSON of ontology terms

Generate enriched terms for human ontology

In [177]:
target_terms = target_graph

In [178]:
len(target_terms)

3

Generate enriched terms for mouse ontology

In [179]:
source_terms = source_graph

In [180]:
len(source_terms)

8

#### Ontology indexing

Create and populate FAISS with human entities

In [181]:
ontology_indexing(ontology_json=TARGET_PATH, faiss_index_path=TARGET_INDEX_PATH, id_tracker_json=TARGET_ID_TRACKER_PATH)

Batches: 100%|██████████| 1/1 [00:00<00:00, 94.34it/s]


Create and populate FAISS with mouse entities

In [182]:
ontology_indexing(ontology_json=SOURCE_PATH, faiss_index_path=SOURCE_INDEX_PATH, id_tracker_json=SOURCE_ID_TRACKER_PATH)

Batches: 100%|██████████| 1/1 [00:00<00:00, 52.74it/s]


#### Execute matching on mouse human pair of ontologies

In [183]:
target_index = faiss.read_index(TARGET_INDEX_PATH)
with open(TARGET_ID_TRACKER_PATH, "r", encoding="utf-8") as f:
    target_id_map = {int(k): v for k, v in json.load(f).items()}

In [184]:
source_index = faiss.read_index(SOURCE_INDEX_PATH)

In [185]:
embs = embed_terms(source_terms, embedding_model)

Batches: 100%|██████████| 1/1 [00:00<00:00, 50.16it/s]


In [186]:
embs_h = embed_terms(target_terms, embedding_model)

Batches: 100%|██████████| 1/1 [00:00<00:00, 90.16it/s]


In [187]:
len(target_id_map)

3

In [188]:
# Step-by-step matching
D, I = faiss_batch_search(embs, target_index, top_k=5)
matches = map_faiss_results(source_terms, D, I, target_id_map)

output_path = "./notebook/result/mouse_to_human_matches_unranked.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(matches, f, indent=2, ensure_ascii=False)

print(f"len match: {len(matches)}")

# Optional refinements
matches = apply_label_reranking(matches)

output_path = "./notebook/result/mouse_to_human_matches_ranked.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(matches, f, indent=2, ensure_ascii=False)

print(f"len match: {len(matches)}")


len match: 8
len match: 8


In [189]:
mouse_index = faiss.read_index(SOURCE_INDEX_PATH)
with open(SOURCE_ID_TRACKER_PATH, "r", encoding="utf-8") as f:
    mouse_id_map = {int(k): v for k, v in json.load(f).items()}

In [190]:
reverse_lookup = precompute_reverse_matches(
    target_terms=target_terms,
    reverse_index=mouse_index,
    reverse_id_map=mouse_id_map,
    model=embedding_model,
    top_k=5
)

Batches: 100%|██████████| 1/1 [00:00<00:00, 99.08it/s]


In [191]:
reverse_lookup_topk = precompute_reverse_matches_topk(
    target_terms=target_terms,
    reverse_index=mouse_index,
    reverse_id_map=mouse_id_map,
    model=embedding_model,
    top_k=3
)

Batches: 100%|██████████| 1/1 [00:00<00:00, 97.74it/s]


In [192]:
len(reverse_lookup)

3

In [193]:
output_path = "./notebook/result/reverse_lookup.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(reverse_lookup, f, indent=2, ensure_ascii=False)

In [194]:
def apply_hcb(matches, reverse_lookup, fallback_threshold=0.95):
    filtered = []
    skipped_top_match = 0
    no_predicited_uri = 0
    for m in matches:
        source_uri = m["source_uri"]
        top_match = m.get("top_match")

        if not top_match:
            skipped_top_match += 1
            continue

        predicted_uri = top_match["uri"]
        confidence = top_match.get("combined_score", top_match.get("score", 0))

        if reverse_lookup.get(predicted_uri) == source_uri:
            filtered.append(m)  # standard HCB
        elif confidence >= fallback_threshold:
            filtered.append(m)  # allow fallback based on confidence
        else: 
            no_predicited_uri += 1

    print(f"skipped_top_match {skipped_top_match}")
    print(f"no_predicited_uri {no_predicited_uri}")

    return filtered


In [195]:
def apply_hcb_with_topk(matches, reverse_lookup_top1, reverse_lookup_topk):
    """
    Filters matches using bidirectional match (HCB), extended to top-k reverse lookup and confidence fallback.
    
    Args:
        matches: list of match dicts (with top_match + score)
        reverse_lookup_top1: dict[target_uri → best source_uri]
        reverse_lookup_topk: dict[target_uri → list of top-k source_uris]
        fallback_threshold: minimum score to allow fallback if HCB fails

    Returns:
        List of filtered matches
    """
    filtered = []
    stats = {
        "strict_hcb": 0,
        "semi_hcb": 0,
        "skipped": 0
    }

    for m in matches:
        source_uri = m["source_uri"]
        top_match = m.get("top_match")

        if not top_match:
            stats["skipped"] += 1
            continue

        predicted_uri = top_match["uri"]

        if reverse_lookup_top1.get(predicted_uri) == source_uri:
            filtered.append(m)
            stats["strict_hcb"] += 1
        elif source_uri in reverse_lookup_topk.get(predicted_uri, []):
            filtered.append(m)
            stats["semi_hcb"] += 1
        else:
            stats["skipped"] += 1

    print("HCB Filtering Summary:")
    for key, count in stats.items():
        print(f"  {key}: {count}")

    return filtered


In [196]:
matches = apply_hcb_with_topk(matches, reverse_lookup, reverse_lookup_topk)

len(matches)

HCB Filtering Summary:
  strict_hcb: 2
  semi_hcb: 3
  skipped: 3


5

In [197]:
output_path = "./notebook_defibrilatorji/result/defibrilatorji_match.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(matches, f, indent=2, ensure_ascii=False)

print(f"Matching complete. Results saved to {output_path}")

Matching complete. Results saved to ./notebook_defibrilatorji/result/defibrilatorji_match.json
